In [10]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import random

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [11]:
# 태그 목록
tags = [
    "병원", "아기 용품", "의학 정보", "태몽", "사주", 
    "별자리 운세", "십이지 운세", "부모 고민 상담", 
    "예방 접종", "육아 보조금", "기타"
]

# 조건 설정
ages = ["20대 후반", "30대 초반", "30대 후반"]
experiences = ["초보", "둘째를 키우는"]
genders = ["엄마", "아빠"]
child_ages = ["0세", "1세", "2세"]
seasons = ["봄", "여름", "가을", "겨울"]
special_situations = ["일반", "쌍둥이", "장애아", "다문화 가정"]

# 비율 설정
age_ratio = [1, 1, 1]
experience_ratio = [1, 1]
gender_ratio = [6, 4]  # 수정된 성별 비율
child_age_ratio = [1, 1, 1]
season_ratio = [1, 1, 1, 1]
special_situation_ratio = [7, 1, 1, 1]

def generate_prompt(tag, age, experience, gender, child_age, season, special_situation):
    if tag == "기타":
        return f"{age}의 일반적인 질문을 생성해주세요. 단, 아이와 관련된 질문이 아니어야 합니다. {season}과 관련된 내용을 포함해주세요."
    
    base_prompt = (
        f"{age} {experience} {gender}가 {child_age} 자녀를 키우고 있습니다. "
        f"현재 계절은 {season}입니다. "
    )
    
    if special_situation != "일반":
        base_prompt += f"이 가정은 {special_situation} 상황입니다. "
    
    base_prompt += (
        f"{child_age} 자녀를 키우는 {gender}가 궁금해할 만한 {tag}에 관한 질문을 하나 생성해주세요. "
        f"{gender}의 입장에서 질문하되, 질문은 30자 내외로 작성해주세요. "
        f"최근에 겪은 특정 상황이나 문제점을 포함하고, 실제 부모가 사용할 법한 자연스러운 표현으로 작성해주세요."
    )
    
    return base_prompt

# 부적절한 태그와 조건 조합 체크
def is_valid_combination(tag, child_age):
    invalid_combos = {
        "태몽": ["1세", "2세"],
        "사주": ["0세"],
        "별자리 운세": ["0세"],
        "십이지 운세": ["0세"]
    }
    return child_age not in invalid_combos.get(tag, [])

In [12]:
# 데이터 프레임 초기화
data = []

# 각 태그별로 100개의 질문 생성
for tag in tags:
    for _ in tqdm(range(100), desc=f"Generating questions for tag: {tag}"):
        while True:
            # 각 비율에 따라 무작위 선택
            age = random.choices(ages, weights=age_ratio, k=1)[0]
            experience = random.choices(experiences, weights=experience_ratio, k=1)[0]
            gender = random.choices(genders, weights=gender_ratio, k=1)[0]
            child_age = random.choices(child_ages, weights=child_age_ratio, k=1)[0]
            season = random.choices(seasons, weights=season_ratio, k=1)[0]
            special_situation = random.choices(special_situations, weights=special_situation_ratio, k=1)[0]
            
            # 유효한 조합인지 확인
            if is_valid_combination(tag, child_age):
                break
        
        # 프롬프트 생성
        prompt = generate_prompt(tag, age, experience, gender, child_age, season, special_situation)
        
        # GPT-3.5-turbo API 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100
        )
        
        # 생성된 질문 추출
        question = response['choices'][0]['message']['content'].strip()
        
        # 데이터 추가
        data.append({
            "tag": tag,
            "age": age,
            "experience": experience,
            "gender": gender,
            "child_age": child_age,
            "season": season,
            "special_situation": special_situation,
            "question": question
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("generated_questions.csv", index=False)

print("Questions generation complete. Saved to 'generated_questions.csv'.")

Generating questions for tag: 의학 정보:  59%|█████▉    | 59/100 [01:07<00:41,  1.02s/it]